In [1]:
from huggingface_hub import notebook_login

In [2]:
notebook_login()
# # Perform the Hugging Face Hub login
# notebook_login(username="your_username", password=getpass.getpass("Enter your password: "))


In [1]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "uz", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "uz", split="test", use_auth_token=True)

print(common_voice)

/home/airi/Makhmud/Whisper/whisper/lib/python3.9/site-packages/datasets/load.py:2483: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/home/airi/Makhmud/Whisper/whisper/lib/python3.9/site-packages/datasets/load.py:1429: FutureWarning: The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Reading metadata...: 47606it [00:00, 194188.22it/s]


Generating validation split: 0 examples [00:00, ? examples/s]

Reading metadata...: 11727it [00:00, 225944.81it/s]


...

Generating other split: 0 examples [00:00, ? examples/s]

Reading metadata...: 126570it [00:00, 216186.97it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]

Reading metadata...: 13448it [00:00, 222176.27it/s]
/home/airi/Makhmud/Whisper/whisper/lib/python3.9/site-packages/datasets/load.py:2483: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/home/airi/Makhmud/Whisper/whisper/lib/python3.9/site-packages/datasets/load.py:1429: FutureWarning: The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 59333
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 12290
    })
})


In [2]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 59333
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 12290
    })
})


In [3]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [4]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Uzbek", task="transcribe")

In [5]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Uzbek", task="transcribe")

In [6]:
print(common_voice["train"][0])

{'audio': {'path': '/home/airi/.cache/huggingface/datasets/downloads/extracted/d8ae35d60dec3978303d984bf8374c0369505fc6be06c0b87646162d487e4622/uz_train_0/common_voice_uz_28907218.mp3', 'array': array([ 4.26325641e-14,  0.00000000e+00,  9.94759830e-14, ...,
       -5.12599363e-05, -8.97714926e-05, -6.41083534e-05]), 'sampling_rate': 48000}, 'sentence': 'Bugun ertalab Gyotenikiga taklifnoma oldim.'}


In [7]:
from IPython.display import Audio

def play_mp3(file_path):
    return Audio(file_path)

for i in range(0, 3):
    print(common_voice["train"][i]['sentence'])
    audio_widget = play_mp3(common_voice["train"][i]['audio']['path'])
    display(audio_widget)
    


Bugun ertalab Gyotenikiga taklifnoma oldim.


Uning badiiy tasvir imkoniyatlarini rivojlantiradi


Udan ko’ra balandroq joy bor.


In [8]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [9]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids

    return batch

In [10]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=4)  

Map (num_proc=4):   0%|          | 0/59333 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/12290 [00:00<?, ? examples/s]

In [6]:
import pickle

In [12]:


# # Save the variable
# with open('common_voice.pickle', 'wb') as f:
#     pickle.dump(common_voice, f)


In [7]:
# Load the variable
with open('common_voice.pickle', 'rb') as f:
    common_voice = pickle.load(f)
common_voice

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 59333
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 12290
    })
})

In [8]:
# common_voice['train'][0]['labels']

In [8]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [9]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [10]:
import evaluate

metric = evaluate.load("wer")
metric 

EvaluationModule(name: "wer", module_type: "metric", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Compute WER score of transcribed segments against references.

Args:
    references: List of references for each speech input.
    predictions: List of transcriptions to score.
    concatenate_texts (bool, default=False): Whether to concatenate all input texts or compute WER iteratively.

Returns:
    (float): the word error rate

Examples:

    >>> predictions = ["this is the prediction", "there is an other sample"]
    >>> references = ["this is the reference", "there is another one"]
    >>> wer = evaluate.load("wer")
    >>> wer_score = wer.compute(predictions=predictions, references=references)
    >>> print(wer_score)
    0.5
""", stored examples: 0)

In [11]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [12]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [11]:
# model 

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
        

In [13]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [14]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-uz",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    # gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    # report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [15]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [16]:
processor.save_pretrained(training_args.output_dir)

In [17]:
trainer.train()

/home/airi/Makhmud/Whisper/whisper/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Wer
1000,0.479400,0.450385,42.072152
2000,0.313000,0.382111,38.939187
3000,0.294800,0.346848,35.427000
4000,0.249000,0.341582,34.928488


Checkpoint destination directory ./whisper-small-uz/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/home/airi/Makhmud/Whisper/whisper/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/airi/Makhmud/Whisper/whisper/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/airi/Makhmud/Whisper/whisper/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was 

TrainOutput(global_step=4000, training_loss=0.5372962049245834, metrics={'train_runtime': 32936.1817, 'train_samples_per_second': 3.886, 'train_steps_per_second': 0.121, 'total_flos': 3.693258236215296e+19, 'train_loss': 0.5372962049245834, 'epoch': 2.16})

In [64]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: uz, split: test",
    "language": "uz",
    "model_name": "Whisper Small Uz - Makhmud Jumanazarov",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

In [59]:
from transformers import WhisperForConditionalGeneration, WhisperTokenizer

# Load the trained model and tokenizer
model = WhisperForConditionalGeneration.from_pretrained("./whisper-small-uz")
tokenizer = WhisperTokenizer.from_pretrained("./whisper-small-uz")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
# trainer.push_to_hub(**kwargs)

In [3]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="Makhmud/whisper-uzbek")  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

iface = gr.Interface(
    fn=transcribe, 
    inputs=gr.Audio(sources="microphone", type="filepath"), 
    outputs="text",
    title="Whisper Small Uzbek",
    description="Realtime demo for Uzbek speech recognition using a fine-tuned Whisper small model.",
)

iface.launch()

AttributeError: partially initialized module 'gradio' has no attribute 'Interface' (most likely due to a circular import)

In [10]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained('./whisper-small-uz')
model.from_pretrained

WhisperForCausalLM(
  (model): WhisperDecoderWrapper(
    (decoder): WhisperDecoder(
      (embed_tokens): Embedding(51865, 768, padding_idx=50257)
      (embed_positions): WhisperPositionalEmbedding(448, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperDecoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (activation_fn): GELUActivation()
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (encoder_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(

In [16]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "./whisper-small-uz"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'path': '0d38672e0bbdbdc460af55b8bb84a15b2730db2819f2af64f9c777d4d586f2de', 'array': array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00024414, 0.00048828,
       0.0005188 ]), 'sampling_rate': 16000}
 Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel. Nor is Mr. Quilter's manner less interesting than his madder. He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similes drawn from eating and its results occur most readily to the mind. He has grave doubts whether Sir Frederick Layton's work is really Greek after all, and can discover in it but little of rocky ethika. Lenel's pictures are a sort of Upguards and Adam paintings, and Mason's exquisite Ittols-Ira's National-A a jingo poem. Mr. Birkut Foster's landscapes smile at one much in the same way that Mr. Karker used to flash his teeth. And Mr. Jon Kodir gives his sitter a cheerful slap in the back, before he says, like a shampoos in a Turkish 

In [38]:
import torchaudio
from pathlib import Path


# Replace the dataset loading code
audio_path = "/home/airi/Makhmud/Whisper/cho-lpon-qor-qo-ynida-lola-hikoya_8WZpEwuV.mp3"

# Load the audio file using torchaudio
waveform, sample_rate = torchaudio.load(audio_path, normalize=True, channels_first=True)

# If the audio has multiple channels, select the first channel
if waveform.shape[0] > 1:
    waveform = waveform[0, :]

# Prepare the input dictionary for the pipeline
input_dict = {"raw": waveform.numpy(), "sampling_rate": sample_rate}

# Perform automatic speech recognition on the local audio file
result = pipe(input_dict)

# Print the recognized text
print(result["text"])


Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


Dam botib, dam chiqib sekingina qirg‘oqqa keldi. Qizlar qichuv bilan ushlab ham oldilar.Suv bilan shalabbo bo‘lg‘on to‘pni, Sharofotxon oldida, huvqochma yo‘lgin, tutilding-ku, deb yerga urdi.Uning bu ishiga Abulla Jallokning qizi To‘txonning achchig‘
